In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

pd.to_datetime('2021-03-31') - pd.to_datetime('2020-10-01')

Timedelta('181 days 00:00:00')

In [2]:
df = pd.read_feather('./data/df.feather')
df.replace([np.inf, -np.inf], np.nan, inplace=True)  # 处理 inf 值
df.fillna(0, inplace=True)
df.head()

,ticker_f,date,30d_return,count,std_7,prc,vol,ret,cap,ret_roll_avg_21d,...,vol_roll_avg_21d,ret_lag_1,ret_lag_2,ret_lag_3,prc_lag_1,prc_lag_2,prc_lag_3,vol_lag_1,vol_lag_2,vol_lag_3
0,A,2020-09-01,0.020486,0.0,0.006449,101.12,1159749.0,0.006971,3.117627e+10,0.000998,...,1.255696e+06,0.005205,0.008887,-0.003121,100.42,99.90,99.02,1077021.0,866211.0,1236929.0
1,A,2020-09-02,0.060392,0.0,0.000000,103.42,1462471.0,0.022745,3.188538e+10,0.002831,...,1.288231e+06,0.006971,0.005205,0.008887,101.12,100.42,99.90,1159749.0,1077021.0,866211.0
2,A,2020-09-03,0.017877,0.0,0.000000,99.64,1992844.0,-0.036550,3.071997e+10,0.000915,...,1.337493e+06,0.022745,0.006971,0.005205,103.42,101.12,100.42,1462471.0,1159749.0,1077021.0
3,A,2020-09-04,-0.006260,0.0,0.000000,96.84,1753874.0,-0.028101,2.985671e+10,-0.000209,...,1.369797e+06,-0.036550,0.022745,0.006971,99.64,103.42,101.12,1992844.0,1462471.0,1159749.0
4,A,2020-09-05,-0.009613,0.0,0.000000,96.84,1753874.0,-0.028101,2.985671e+10,-0.000209,...,1.369797e+06,-0.036550,0.022745,0.006971,99.64,103.42,101.12,1992844.0,1462471.0,1159749.0


In [4]:
df.ticker_f.value_counts()

ticker_f
ITUB     393
NTAP     393
OMC      393
OKE      393
O        393
        ... 
EMB      393
EL       393
EIX      393
A        391
DISCB    382
Name: count, Length: 466, dtype: int64

In [12]:
X, y, date_mask = [], [], []

features =['count', 'std_7', 'prc', 'vol', 'cap', 'ret',]
step = 7
start_date = '2020-10-01'
end_date = '2021-04-30'

for ticker_f, data in tqdm(df.groupby('ticker_f')):
    data.sort_values('date', inplace=True)
    
    for i in range(step, len(data), 1):
        _X = data.iloc[i-step:i, data.columns.isin(features)].values
        _y = int(data.iloc[i, data.columns.isin(['30d_return'])].item() >= 0)
        _date_mask = (data.iloc[i, data.columns.get_loc('date')] - pd.to_datetime(start_date)).days

        X.append(_X)
        y.append(_y)
        date_mask.append(_date_mask)
        

X = np.array(X)
y = np.array(y)
date_mask = np.array(date_mask)

print(X.shape, y.shape, date_mask.shape)

100%|██████████| 466/466 [02:13<00:00,  3.49it/s]


(179863, 7, 6) (179863,) (179863,)


In [15]:
data = {
    "X": X,
    "y": y,
    "date_mask": date_mask,
}

import pickle
with open('./data/data.pkl', 'wb') as f:
    pickle.dump(data, f)

In [ ]:
X_train, y_train = [], []
X_test, y_test = [], []

features =['count', 'std_7', 'prc', 'vol', 'cap', 'ret',]
step = 7
split_date = '2021-07-01'

for ticker_f, data in tqdm(df.groupby('ticker_f')):
    data.sort_values('date', inplace=True)
    
    for i in range(step, len(data), 1):
        _X = data.iloc[i-step:i, data.columns.isin(features)].values
        _y = int(data.iloc[i, data.columns.isin(['30d_return'])].item() >= 0)
        
        if data.iloc[i, data.columns.get_loc('date')] < pd.to_datetime(split_date):
            X_train.append(_X)
            y_train.append(_y)
        else:
            X_test.append(_X)
            y_test.append(_y)
        
    # break

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [12]:
data = {
    "X_train": X_train,
    "X_test": X_test,
    "y_train": y_train,
    "y_test": y_test
}

import pickle
with open('./data/data.pkl', 'wb') as f:
    pickle.dump(data, f)
